# Classification Predict Student Solution

© Explore Data Science Academy

---
### Honour Code

I {MASHAKO JUSTICE MANYELO}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

# Predict Overview: Climate Change Sentiment Analysis

## Project Overview



# Table of Contents

1. [Project Overview](#project-overview)
2. [Step 1: Import Necessary Libraries](#step-1-import-necessary-libraries)
3. [Step 2: Load the Dataset](#step-2-load-the-dataset)
4. [Step 3: Explore the Dataset](#step-3-explore-the-dataset)
5. [Step 4: Preprocess the Text Data](#step-4-preprocess-the-text-data)
6. [Step 5: Split the Dataset into Training and Testing Sets](#step-5-split-the-dataset-into-training-and-testing-sets)
7. [Step 6: Build a Basic Classification Model](#step-6-build-a-basic-classification-model)
8. [Step 7: Evaluate the Model on the Test Set](#step-7-evaluate-the-model-on-the-test-set)
9. [Step 8: Cross-Validation for Model Evaluation](#step-8-cross-validation-for-model-evaluation)
10. [Step 9: Predict Sentiments for the Provided Test Dataset](#step-9-predict-sentiments-for-the-provided-test-dataset)
11. [Step 10: Prepare the Submission File](#step-10-prepare-the-submission-file)

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    
| Description: Importing Packages |
| :--------------------------- |
| In this section you are required to import, and briefly discuss, the libraries that will be used throughout your analysis and modelling. |

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
import time
%matplotlib inline
from matplotlib.colors import ListedColormap
import seaborn as sns
import re
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from comet_ml import Experiment
from comet_ml.integration.sklearn import log_model
from comet_ml import Experiment
from sklearn.metrics import confusion_matrix, classification_report
from nltk.stem import PorterStemmer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


<a id="two"></a>
## Step 2: Load the Dataset
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| Description: Loading the data |
| :--------------------------- |
| Replace 'your_dataset.csv' with the actual file path or URL. |

---

In [2]:
# Load training data
train_df = pd.read_csv("train_set")

# Load test data without labels
test_df = pd.read_csv("test_set")

FileNotFoundError: [Errno 2] No such file or directory: 'train_set'

<a id="three"></a>
## Step 3: Explore the Dataset
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| Description: Exploratory data analysis |
| :--------------------------- |
| Display basic information about the dataset. |

---


<a id="four"></a>
## Step 4: Preprocess the Text Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| Description: Data engineering |
| :--------------------------- |
| Perform text cleaning, handling missing values, etc... |

---

<a id="five"></a>
## Step 5: Split the Dataset into Training and Testing Sets
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| Description: Modelling |
| :--------------------------- |
| In this step, we divide our dataset into two subsets: a training set and a testing set. |

---

<a id="six"></a>
## Step 6: Build a Basic Classification Model
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| Description: Model performance |
| :--------------------------- |
| Start with a simple model. We will add more as we go. |

---

<a id="seven"></a>
## Step 7: Evaluate the Model on the Test Set
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| Description: Model explanation |
| :--------------------------- |
| In this section, you are required to discuss how the best performing model works in a simple way. |

---

<a id="seven"></a>
## Step 8: Cross-Validation for Model Evaluation
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| Description: Model explanation |
| :--------------------------- |
| Ensure the model's robustness. |

---

<a id="seven"></a>
## Step 9: Predict Sentiments for the Provided Test Dataset
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| Description: Model explanation |
| :--------------------------- |
| Replace 'your_test_dataset.csv' with the actual file path or URL. |

---

<a id="seven"></a>
## Step 10: Prepare the Submission File
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| Description: Model explanation |
| :--------------------------- |
| In this final step, we prepare the submission file that aligns with the requirements for the project. |

---